<a href="https://colab.research.google.com/github/Maan2911/INT248-CA1-Face-Mask-Detection/blob/main/Face_Mask_Detection_CA1_INT248.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import zipfile
with zipfile.ZipFile('/content/drive/My Drive/Face Mask Detection.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/drive/My Drive/')

In [5]:
%cd /content/drive/My\ Drive/Face Mask Detection

/content/drive/My Drive/Face Mask Detection


In [8]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import argparse
import cv2
import os
from google.colab.patches import cv2_imshow
from natsort import natsorted, ns

In [11]:
def plotCorrelationMatrix(df, graphWidth):
    filename = df.dataframeName
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()


In [12]:
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()

In [17]:
class_name = ['Incorrect','With_Mask','Without_Mask']

In [23]:
train_gen=ImageDataGenerator(
rescale=1/255,
validation_split=0.2)

test_gen=ImageDataGenerator(rescale=1./255)

In [24]:
input=layers.Input(shape=(224,224,3))

x=layers.Conv2D(16,3,activation="relu")(input)
x=layers.BatchNormalization()(x)
x=layers.MaxPooling2D((2,2))(x)


x=layers.Conv2D(32,3,activation="relu")(x)
x=layers.BatchNormalization()(x)
x=layers.MaxPooling2D((2,2))(x)


x=layers.Flatten()(x)
x=layers.Dense(64,activation="relu")(x)
x=layers.Dropout(0.3)(x)
x=layers.Dense(32,activation="relu")(x)
output=layers.Dense(1,activation="sigmoid")(x)

model=Model(input,output)

model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 222, 222, 16)      448       
                                                                 
 batch_normalization (BatchN  (None, 222, 222, 16)     64        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 32)      4640      
                                                                 
 batch_normalization_1 (Batc  (None, 109, 109, 32)     128   

In [25]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [ ]:
mask_dict={
    'incorrect': list(data_dir.glob('mask_weared_incorrect/*')),
    'with_mask': list(data_dir.glob('with_mask/*')),
    'without_mask': list(data_dir.glob('without_mask/*')),
}

In [ ]:
mask_labels={
    'incorrect': 0,
    'with_mask': 1,
    'without_mask': 2,
}

In [ ]:
x, y=[],[]
for mask_name, images in mask_dict.items():
    for image in images:
        img=cv2.imread(str(image))
        x.append(img)
        y.append(mask_labels[mask_name])

In [ ]:
len(x)

In [ ]:
class_labels=[0,1,2,3]
import sklearn.metrics as metrics
report = metrics.classification_report(y_test, pred_classes, class_labels)
print(report) 
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred_classes)